In [1]:
## CHANGE THIS TO SHOW LOADING OF RELEVANT CLEANED DATA SETS
VTR2014 <- read.csv("C:/Users/Allan/Downloads/VTR2014.txt")
#loads the relevant data
original_count <- length(VTR2014$V_VIN)
original_count <- as.data.frame(original_count)



Warning message in file(file, "rt"):
"cannot open file 'C:/Users/Allan/Downloads/VTR2014.txt': No such file or directory"

ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:
VTR2014 <- transform(VTR2014, HC_adjusted = VTR2014$V_HC)
VTR2014 <- transform(VTR2014, CO_adjusted = VTR2014$V_CO)
VTR2014 <- transform(VTR2014, NOX_adjusted = VTR2014$V_NOX)



In [ ]:
## PREVIOUS DATA CLEANING STEPS; SUMMARIZE AND DELETE THIS FROM PROJECT

#finds the original number of vehicles in the data
VTR2014 <- VTR2014[,c("V_VIN","V_DATE_TIME" ,"V_MAKE", "V_MODEL", "V_VEH_YEAR", "V_ODOMETER" , "HC_adjusted", "CO_adjusted", "NOX_adjusted", "V_CO_STD", "V_HC_STD", "V_NOX_STD", "V_OBD_RES", "V_EM_RES", "V_RESULT" , "V_TRANS", "V_CYLINDERS", "V_DISP", "V_DRIVE", "V_CO_RES", "V_HC_RES", "V_NOX_RES" )]
#cuts down data to only the relevant fields

VIN_cleaned <- trimws(VTR2014$V_VIN,"both")
#makes a VIN category wihtout any white space beofre or after the VIN

VTR2014 <- transform(VTR2014, VIN_cleaned = VIN_cleaned)
#appends the cleaned vin category to the original dataset

MAKE_cleaned <- trimws(VTR2014$V_MAKE,"both")
#makes a car make category wihtout any white space beofre or after the VIN

VTR2014 <- transform(VTR2014, MAKE_cleaned = MAKE_cleaned)
#appends the cleaned make category to the original dataset

MODEL_cleaned <- trimws(VTR2014$V_MODEL,"both")
#makes a model category wihtout any white space beofre or after the VIN

VTR2014 <- transform(VTR2014, MODEL_cleaned = MODEL_cleaned)
#appends the cleaned model category to the original dataset


VTR2014 <- VTR2014[nchar(as.character(VTR2014$VIN_cleaned)) == 17 ,]
#filters the data so only rows with a 17 digit cleaned_vin show up
digit_17_VIN_count <- length(VTR2014$V_VIN)
digit_17_VIN_count <- as.data.frame(digit_17_VIN_count)
mini_vin <- substr(VTR2014$VIN_cleaned, 2 ,4)
mini_vin <-paste(mini_vin, substr(VTR2014$VIN_cleaned, 8,8), sep = "")
mini_vin <-paste(mini_vin, substr(VTR2014$VIN_cleaned, 10,10), sep = "")
mini_vin <- as.data.frame(mini_vin)
#creates new field of truncated VIN numbers

VTR2014 <- cbind(VTR2014, mini_vin)
#makes new data set with the new field in it

VTR2014 <- subset(VTR2014, !is.na(HC_adjusted & CO_adjusted & NOX_adjusted))
#VTR_VIN_NA_check <- VTR2014[!is.na(VTR2014$V_HC) & !is.na(VTR2014$V_CO) & !is.na(VTR2014$V_NOX),]
#all.equal(VTR2014, VTR_VIN_NA_check)
#filters out all NA values for HC, Co, NOX

VTR2014 <- VTR2014[VTR2014$HC_adjusted > 0 & VTR2014$CO_adjusted > 0 & VTR2014$NOX_adjusted > 0 & VTR2014$V_VEH_YEAR < 2014,]
#gets rid of data that has all zeros for emissions results
non_zero_emissions_count <- length(VTR2014$V_VIN)
non_zero_emissions_count <- as.data.frame(non_zero_emissions_count)

sorted_list <- VTR2014[with(VTR2014, order(VIN_cleaned, V_DATE_TIME, decreasing=FALSE)),] #no quotes around column names
#sort the data by date and vin

first_tests <- sorted_list[!duplicated(sorted_list$VIN_cleaned, fromLast=FALSE),]
last_tests <- sorted_list[!duplicated(sorted_list$VIN_cleaned, fromLast=TRUE),]
#get entries from unique vins for first tests only and last tests only

count_of_first_tests <- length(first_tests$V_VIN)
count_of_first_tests <- as.data.frame(count_of_first_tests)

#VTR2014 <- first_tests[first_tests$V_EM_RES == "P" | first_tests$V_EM_RES == "F",]
#filters results so only clean IM240 results are shown
count_of_IM240_clean <- length(VTR2014$V_VIN)
count_of_IM240_clean <- as.data.frame(count_of_IM240_clean)



In [ ]:
#creates a new dataframe that includes only records for cars that eventually passed the emissions test
#for these cars, the first and last (if applicable) emissions quantities found during testing were added as columns
first_tests_new <- transform(first_tests, new_CO = last_tests$CO_adjusted)
first_tests_new <- transform(first_tests_new, new_NOX = last_tests$NOX_adjusted)
first_tests_new <- transform(first_tests_new, new_HC = last_tests$HC_adjusted)
first_tests_new <- transform(first_tests_new, last_CO = last_tests$V_CO_RES)
first_tests_new <- transform(first_tests_new, last_HC = last_tests$V_HC_RES)
first_tests_new <- transform(first_tests_new, last_NOX = last_tests$V_NOX_RES)
first_tests_new <- transform(first_tests_new, last_time = last_tests$V_DATE_TIME)



In [ ]:

#standardized time, taking it out of the original dataframe's format so timestamps can be subtracted
first_tests_new$V_DATE_TIME <- as.POSIXct(first_tests_new$V_DATE_TIME, tz = "GMT")
first_tests_new$last_time <- as.POSIXct(first_tests_new$last_time, tz = "GMT")



In [ ]:
#new field created, to measure the delta in time between the initial emissions test and the final emissions test for a given vehicle
first_tests_new <- transform(first_tests_new, delta_time = first_tests_new$last_time - first_tests_new$V_DATE_TIME)
first_tests_new <- transform(first_tests_new, new_delta = as.numeric(first_tests_new$delta_time))


In [ ]:
#filters the dataset to only include cars with failing tests
first_tests_new <- first_tests_new[first_tests_new$V_EM_RES =="F",]
first_tests_new <- first_tests_new[first_tests_new$new_delta > 1,]



In [ ]:
first_tests_new <- transform(first_tests_new, mileage_per_year = first_tests_new$V_ODOMETER/(2014 - first_tests_new$V_VEH_YEAR))
#creates new data set that holds both initial and final emissions results. Also creates a mileage per year field



In [ ]:
#creates a new field, showing the difference between initial emissions and final emissions in a vehicle. this difference is plotted
first_tests_new <- transform(first_tests_new, plotted_first_difference_CO = first_tests_new$CO_adjusted - first_tests_new$V_CO_STD)
first_tests_new <- transform(first_tests_new, plotted_last_difference_CO = first_tests_new$new_CO - first_tests_new$V_CO_STD)
plot(first_tests_new$plotted_first_difference_CO, first_tests_new$plotted_last_difference_CO)




In [ ]:
#the same process done above is repeated for all emissions tested for the vehicles, and plotted
#first_tests_CO <- first_tests_new[first_tests_new$CO_adjusted - first_tests_new$new_CO > 0 & first_tests_new$last_CO == "P",]
first_tests_CO <- first_tests_CO[first_tests_CO$V_CO_RES == "F",]
first_tests_NOX <- first_tests_new[first_tests_new$NOX_adjusted - first_tests_new$new_NOX > 0 & first_tests_new$last_NOX == "P",]
first_tests_HC <- first_tests_new[first_tests_new$HC_adjusted - first_tests_new$new_HC > 0 & first_tests_new$last_HC == "P",]
#creates new data sets to hold cars that failed a particular pollutant test


In [ ]:

first_tests_CO <- transform(first_tests_CO, CO_saved = first_tests_CO$mileage_per_year*(first_tests_CO$CO_adjusted - first_tests_CO$new_CO)/907185 )
first_tests_NOX <- transform(first_tests_NOX, NOX_saved = first_tests_NOX$mileage_per_year*(first_tests_NOX$NOX_adjusted - first_tests_NOX$new_NOX)/907185 )
first_tests_HC <- transform(first_tests_HC, HC_saved = first_tests_HC$mileage_per_year*(first_tests_HC$HC_adjusted - first_tests_HC$new_HC)/907185 )
#finds how much emissions were saved with each vehicle



In [ ]:
#emissions saved due to testing is calculated below
#this is done as follows:
#multiply the average milage travelled for the vehicle by the delta in emissions found above.
CO_saved_by_mini_vin <- aggregate(CO_saved~mini_vin, first_tests_CO, sum)
CO_saved_by_mini_vin <- CO_saved_by_mini_vin[with(CO_saved_by_mini_vin, order(CO_saved, decreasing = TRUE)),]
#finds the CO saved by miniVIN

Nox_saved_by_mini_vin <- aggregate(NOX_saved~mini_vin, first_tests_NOX, sum)
Nox_saved_by_mini_vin <- Nox_saved_by_mini_vin[with(Nox_saved_by_mini_vin, order(NOX_saved, decreasing = TRUE)),]
#finds the NOX saved by minivin

HC_saved_by_mini_vin <- aggregate(HC_saved~mini_vin, first_tests_HC, sum)
HC_saved_by_mini_vin <- HC_saved_by_mini_vin[with(HC_saved_by_mini_vin, order(HC_saved, decreasing = TRUE)),]
#finds the HC saved by miniVIN



In [ ]:
#the above values, initially per mini_vin, are aggregated to find total emissions saved due to emissions testing below.
total_CO_saved <- sum(CO_saved_by_mini_vin$CO_saved)
total_NOX_saved <- sum(Nox_saved_by_mini_vin$NOX_saved)
total_HC_saved <- sum(HC_saved_by_mini_vin$HC_saved)
#finds the total emissions saved




In [ ]:
#pollutants lost shown below.

#same procedure done as above, but for vehicles that failed their last emissions test
#this helps identify potential emissions lost due to cars that continue driving for a period after failing inspection

first_tests_CO_failed_last <- first_tests_new[first_tests_new$CO_adjusted - first_tests_new$new_CO < 0 & first_tests_new$last_CO == "F",]
first_tests_NOX_failed_last <- first_tests_new[first_tests_new$NOX_adjusted - first_tests_new$new_NOX < 0 & first_tests_new$last_CO == "F",]
first_tests_HC_failed_last <- first_tests_new[first_tests_new$HC_adjusted - first_tests_new$new_HC < 0 & first_tests_new$last_CO == "F",]
#creates new data sets to hold cars that failed a particular pollutant test

first_tests_CO_failed_last <- transform(first_tests_CO_failed_last, CO_lost = first_tests_CO_failed_last$mileage_per_year*(first_tests_CO_failed_last$CO_adjusted - first_tests_CO_failed_last$new_CO)/907185 )
first_tests_NOX_failed_last <- transform(first_tests_NOX_failed_last, NOX_lost = first_tests_NOX_failed_last$mileage_per_year*(first_tests_NOX_failed_last$NOX_adjusted - first_tests_NOX_failed_last$new_NOX)/907185 )
first_tests_HC_failed_last <- transform(first_tests_HC_failed_last, HC_lost = first_tests_HC_failed_last$mileage_per_year*(first_tests_HC_failed_last$HC_adjusted - first_tests_HC_failed_last$new_HC)/907185 )
#finds how much emissions were saved with each vehicle

CO_lost_by_mini_vin <- aggregate(CO_lost~mini_vin, first_tests_CO_failed_last, sum)
CO_lost_by_mini_vin <- CO_lost_by_mini_vin[with(CO_lost_by_mini_vin, order(CO_lost, decreasing = TRUE)),]
#finds the CO saved by miniVIN


HC_lost_by_mini_vin <- aggregate(HC_lost~mini_vin, first_tests_HC_failed_last, sum)
HC_lost_by_mini_vin <- HC_lost_by_mini_vin[with(HC_lost_by_mini_vin, order(HC_lost, decreasing = TRUE)),]
#finds the CO saved by miniVIN

NOX_lost_by_mini_vin <- aggregate(NOX_lost~mini_vin, first_tests_NOX_failed_last, sum)
NOX_lost_by_mini_vin <- NOX_lost_by_mini_vin[with(NOX_lost_by_mini_vin, order(NOX_lost, decreasing = TRUE)),]
#finds the CO saved by miniVIN

total_CO_lost <- sum(CO_lost_by_mini_vin$CO_lost)
total_NOX_lost <- sum(NOX_lost_by_mini_vin$NOX_lost)
total_HC_lost <- sum(HC_lost_by_mini_vin$HC_lost)

